<a href="https://www.kaggle.com/code/vijays140291/rating-prediction?scriptVersionId=153814329" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#importing the library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso

In [2]:
#loading the datatset
df=pd.read_csv('/kaggle/input/80-cereals/cereal.csv')
#loading the dataset
df

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,60,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12,25,25,2,1.0,1.00,27.753301
74,Wheat Chex,R,C,100,3,1,230,3.0,17.0,3,115,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,110,25,1,1.0,1.00,51.592193


In [3]:
#getting the preliminary information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      77 non-null     object 
 1   mfr       77 non-null     object 
 2   type      77 non-null     object 
 3   calories  77 non-null     int64  
 4   protein   77 non-null     int64  
 5   fat       77 non-null     int64  
 6   sodium    77 non-null     int64  
 7   fiber     77 non-null     float64
 8   carbo     77 non-null     float64
 9   sugars    77 non-null     int64  
 10  potass    77 non-null     int64  
 11  vitamins  77 non-null     int64  
 12  shelf     77 non-null     int64  
 13  weight    77 non-null     float64
 14  cups      77 non-null     float64
 15  rating    77 non-null     float64
dtypes: float64(5), int64(8), object(3)
memory usage: 9.8+ KB


In [4]:
#Preprocessing the dataset
df.isna().sum()

name        0
mfr         0
type        0
calories    0
protein     0
fat         0
sodium      0
fiber       0
carbo       0
sugars      0
potass      0
vitamins    0
shelf       0
weight      0
cups        0
rating      0
dtype: int64

In [5]:
(df==-1).sum()

name        0
mfr         0
type        0
calories    0
protein     0
fat         0
sodium      0
fiber       0
carbo       1
sugars      1
potass      2
vitamins    0
shelf       0
weight      0
cups        0
rating      0
dtype: int64

In [6]:
#replacing -1 with Nan
df=df.replace(-1,np.NaN)

In [7]:
for column in ['carbo','sugars','potass']:
    df[column]=df[column].fillna(df[column].mean())

In [8]:
(df==-1).sum()

name        0
mfr         0
type        0
calories    0
protein     0
fat         0
sodium      0
fiber       0
carbo       0
sugars      0
potass      0
vitamins    0
shelf       0
weight      0
cups        0
rating      0
dtype: int64

In [9]:
{column:list(df[column].unique()) for column in ['mfr','type']}

{'mfr': ['N', 'Q', 'K', 'R', 'G', 'P', 'A'], 'type': ['C', 'H']}

In [10]:
df['type']=df['type'].replace({'C':0,'H':1})

In [11]:
df['type'].unique()

array([0, 1])

In [12]:
dummies=pd.get_dummies(df['mfr'])

In [13]:
df=pd.concat([df,dummies],axis=1)

In [14]:
df=df.drop('mfr',axis=1)

In [15]:
df

,name,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,...,weight,cups,rating,A,G,K,N,P,Q,R
0,100% Bran,0,70,4,1,130,10.0,5.0,6.0,280.000000,...,1.0,0.33,68.402973,False,False,False,True,False,False,False
1,100% Natural Bran,0,120,3,5,15,2.0,8.0,8.0,135.000000,...,1.0,1.00,33.983679,False,False,False,False,False,True,False
2,All-Bran,0,70,4,1,260,9.0,7.0,5.0,320.000000,...,1.0,0.33,59.425505,False,False,True,False,False,False,False
3,All-Bran with Extra Fiber,0,50,4,0,140,14.0,8.0,0.0,330.000000,...,1.0,0.50,93.704912,False,False,True,False,False,False,False
4,Almond Delight,0,110,2,2,200,1.0,14.0,8.0,98.666667,...,1.0,0.75,34.384843,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,0,110,2,1,250,0.0,21.0,3.0,60.000000,...,1.0,0.75,39.106174,False,True,False,False,False,False,False
73,Trix,0,110,1,1,140,0.0,13.0,12.0,25.000000,...,1.0,1.00,27.753301,False,True,False,False,False,False,False
74,Wheat Chex,0,100,3,1,230,3.0,17.0,3.0,115.000000,...,1.0,0.67,49.787445,False,False,False,False,False,False,True
75,Wheaties,0,100,3,1,200,3.0,17.0,3.0,110.000000,...,1.0,1.00,51.592193,False,True,False,False,False,False,False


In [16]:
for column in df.columns[-7:]:
    df[column]=df[column].astype(int)

In [17]:
df

,name,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,...,weight,cups,rating,A,G,K,N,P,Q,R
0,100% Bran,0,70,4,1,130,10.0,5.0,6.0,280.000000,...,1.0,0.33,68.402973,0,0,0,1,0,0,0
1,100% Natural Bran,0,120,3,5,15,2.0,8.0,8.0,135.000000,...,1.0,1.00,33.983679,0,0,0,0,0,1,0
2,All-Bran,0,70,4,1,260,9.0,7.0,5.0,320.000000,...,1.0,0.33,59.425505,0,0,1,0,0,0,0
3,All-Bran with Extra Fiber,0,50,4,0,140,14.0,8.0,0.0,330.000000,...,1.0,0.50,93.704912,0,0,1,0,0,0,0
4,Almond Delight,0,110,2,2,200,1.0,14.0,8.0,98.666667,...,1.0,0.75,34.384843,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,0,110,2,1,250,0.0,21.0,3.0,60.000000,...,1.0,0.75,39.106174,0,1,0,0,0,0,0
73,Trix,0,110,1,1,140,0.0,13.0,12.0,25.000000,...,1.0,1.00,27.753301,0,1,0,0,0,0,0
74,Wheat Chex,0,100,3,1,230,3.0,17.0,3.0,115.000000,...,1.0,0.67,49.787445,0,0,0,0,0,0,1
75,Wheaties,0,100,3,1,200,3.0,17.0,3.0,110.000000,...,1.0,1.00,51.592193,0,1,0,0,0,0,0


In [18]:
#Split and Scaling the dataset
y=df.loc[:,'rating']
x=df.drop('rating',axis=1)

In [19]:
x=x.drop('name',axis=1)
x

,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,A,G,K,N,P,Q,R
0,0,70,4,1,130,10.0,5.0,6.0,280.000000,25,3,1.0,0.33,0,0,0,1,0,0,0
1,0,120,3,5,15,2.0,8.0,8.0,135.000000,0,3,1.0,1.00,0,0,0,0,0,1,0
2,0,70,4,1,260,9.0,7.0,5.0,320.000000,25,3,1.0,0.33,0,0,1,0,0,0,0
3,0,50,4,0,140,14.0,8.0,0.0,330.000000,25,3,1.0,0.50,0,0,1,0,0,0,0
4,0,110,2,2,200,1.0,14.0,8.0,98.666667,25,3,1.0,0.75,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0,110,2,1,250,0.0,21.0,3.0,60.000000,25,3,1.0,0.75,0,1,0,0,0,0,0
73,0,110,1,1,140,0.0,13.0,12.0,25.000000,25,2,1.0,1.00,0,1,0,0,0,0,0
74,0,100,3,1,230,3.0,17.0,3.0,115.000000,25,1,1.0,0.67,0,0,0,0,0,0,1
75,0,100,3,1,200,3.0,17.0,3.0,110.000000,25,1,1.0,1.00,0,1,0,0,0,0,0


In [20]:
scaler=StandardScaler()
x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [21]:
#Splitting between train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)

In [22]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(53, 20)
(24, 20)
(53,)
(24,)


In [23]:
model=LinearRegression()
model_lr=Ridge(alpha=1.0)
model_lasso=Lasso(alpha=1.0)


In [24]:
for model in [model,model_lr,model_lasso]:
    model.fit(x_train,y_train)
    print(model,model.score(x_test,y_test))

LinearRegression() 0.9513268469599576
Ridge() 0.9783320287542778
Lasso() 0.949114496820733


In [25]:
x_train

,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,A,G,K,N,P,Q,R
5,-0.201347,0.161019,-0.501495,0.987096,0.244034,-0.275335,-1.115756e+00,6.881272e-01,-0.415306,-0.14627,-1.460273,-0.198067,-0.307262,-0.114708,1.581139,-0.652630,-0.290701,-0.363803,-0.340503,-0.340503
34,-0.201347,0.677623,0.417912,1.987181,-1.016680,0.358155,-4.674575e-01,-7.003064e-01,0.019317,-0.14627,0.957813,-0.198067,-2.123870,-0.114708,-0.632456,-0.652630,-0.290701,2.748737,-0.340503,-0.340503
66,-0.201347,0.161019,-0.501495,-0.012988,-1.076714,-0.486498,-1.504735e+00,1.845155e+00,-0.849929,-0.14627,-0.251230,-0.198067,-0.307262,-0.114708,-0.632456,1.532262,-0.290701,-0.363803,-0.340503,-0.340503
69,-0.201347,0.161019,-0.501495,-0.012988,0.484170,-0.908824,1.607098e+00,-9.317120e-01,-0.922366,3.23257,0.957813,-0.198067,0.774053,-0.114708,1.581139,-0.652630,-0.290701,-0.363803,-0.340503,-0.340503
67,-0.201347,0.161019,3.176134,-1.013072,0.844374,-0.486498,3.105010e-01,-9.317120e-01,-0.632618,-0.14627,-1.460273,-0.198067,0.774053,-0.114708,-0.632456,1.532262,-0.290701,-0.363803,-0.340503,-0.340503
51,-0.201347,1.194228,0.417912,0.987096,0.123966,-0.275335,-3.377977e-01,6.881272e-01,0.309065,-0.14627,0.957813,1.474213,-1.388576,-0.114708,1.581139,-0.652630,-0.290701,-0.363803,-0.340503,-0.340503
53,-0.201347,-0.355585,0.417912,-1.013072,1.924986,-0.486498,1.347779e+00,-9.317120e-01,-0.777492,3.23257,0.957813,-0.198067,0.774053,-0.114708,-0.632456,1.532262,-0.290701,-0.363803,-0.340503,-0.340503
33,-0.201347,0.161019,0.417912,-1.013072,0.123966,0.358155,5.698204e-01,-9.317120e-01,-0.125558,-0.14627,0.957813,-0.198067,-2.469891,-0.114708,-0.632456,-0.652630,-0.290701,2.748737,-0.340503,-0.340503
40,-0.201347,0.161019,-0.501495,-0.012988,1.204578,-0.908824,1.607098e+00,-9.317120e-01,-0.849929,-0.14627,-0.251230,-0.198067,2.936682,-0.114708,1.581139,-0.652630,-0.290701,-0.363803,-0.340503,-0.340503
35,-0.201347,0.677623,-1.420902,0.987096,0.724306,-0.486498,-7.267769e-01,9.195328e-01,-0.777492,-0.14627,-0.251230,-0.198067,0.774053,-0.114708,-0.632456,-0.652630,-0.290701,-0.363803,2.936835,-0.340503
